# Summary
This document corresponds to Exercise 4 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Course.pdf).

If [PEPit](https://pypi.org/project/PEPit/) is not already installed, please execute the following cell.

In [ ]:
!pip install pepit

### Exercise 4.1

This exercise requires no coding.

### Exercise 4.2

The following lines allow computing the worst-case value of the ratio $\frac{f(x_N)-f_\star}{\|x_0-x_\star\|^2}$ for the proximal point algorithm when $f$ is a (closed, proper) convex function.

In [ ]:
from PEPit import PEP
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step

def wc_proximal_point(gamma, n, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a convex function
    func = problem.declare_function(ConvexFunction)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Run n steps of the proximal point method
    x = x0
    for _ in range(n):
        x, _, fx = proximal_step(x, func, gamma)

    # Set the performance metric to the final distance to optimum in function values
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose)

    # Return the worst-case guarantee of the evaluated method 
    return pepit_tau

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

n_max = 30
n_list = list(range(1,n_max))
gamma = 1
verbose = 0

pepits_taus_ppm = list()

for i in range(n_max-1):
    t0= time.process_time()
    pepit_tau,_ = wc_proximal_point(gamma, i+1, verbose)
    pepits_taus_ppm.append(pepit_tau)
    t1 = time.process_time() - t0
    print(i+1, '/', n_max-1,' done (elapsed time:',"%.2f" %t1,'[s])')

In [ ]:
plt.plot(n_list, pepits_taus_ppm, '-')

plt.xlabel('N')
plt.ylabel('f(x_i)-f_\star')
plt.loglog()
plt.show()

### Exercise 4.3

Using the previous code, can you guess the dependence (of the value of the worst-case ratio) on $N$ and $\gamma$?

### Exercise 4.4

Can you find a low-dimensional worst-case example?

The following cells use the trace heuristic for trying to identify a low-dimensional worst-case example.

In [ ]:
from PEPit import PEP
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step

def wc_proximal_point(gamma, n, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a convex function
    func = problem.declare_function(ConvexFunction)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Run n steps of the proximal point method
    x = x0
    list_x = list() # store all x's
    list_f = list() # store all f's
    for _ in range(n):
        x, _, fx = proximal_step(x, func, gamma)
        list_x.append(x)
        list_f.append(fx)

    # Set the performance metric to the final distance to optimum in function values
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose,dimension_reduction_heuristic="trace")
    
    # Store all the points
    list_x_solved = list()
    list_f_solved = list()
    for i in range(n):
        list_x_solved.append((list_x[i]-xs).eval())
        list_f_solved.append((list_f[i]-fs).eval())

    # Return the worst-case guarantee of the evaluated method 
    return pepit_tau, list_x_solved, list_f_solved

Let's test this code: what's the output for certain values of $\gamma$ and $N$? What's the dimension of the worst-case found?

In [ ]:
gamma = 1
n = 20

# compute a low-dimensional worst-case example
pepit_tau, list_x_solved, list_f_solved = wc_proximal_point(gamma, n, verbose=1)

Plot the one-dimensional worst-case example!

In [ ]:
import matplotlib.pyplot as plt

# if there is only 1 nonzero eigenvalue, plot the 1-dimensional WC function:
first_coordinate_x = list()
for i in range(n):
    first_coordinate_x.append(list_x_solved[i][0])
    
plt.plot(first_coordinate_x,list_f_solved,'.',label='iterates')
plt.plot(first_coordinate_x[0],list_f_solved[0],'.',label='x0')
plt.plot(0,0,'.',label='xs')
plt.legend()